In [1]:
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn import preprocessing, model_selection

In [2]:
#Description of the feratures
'''
Pclass Passenger Class(1=1st, 2= 2nd, 3=3rd)
survival Survival(0=No; 1=Yes)
name Name
sex Sex
sibsp Number of Siblings/Spouse Abroad
parch Number of parent/children Abroad
ticket Ticket number
fare Passenger Fare (British Pound)
cabin Cabin
embarked Port of Embarkation(C=Cherborug; Q= queens town; S= Southhampton)
boat Lifeboat
body Body Identification Number
home.dest Home/Identification Number
'''
df = pd.read_excel(r'C:\Users\pc\Downloads\titanic.xls', delim_whitespace=True)
df.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [3]:
#Now, I want to drop certain columns that are not reasonable
df.drop(['name','body'], axis=1, inplace=True)
#df.convert_objects(convert_numeric=True)
df.fillna(0, inplace=True)
df.head()

,pclass,survived,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,home.dest
0,1,1,female,29.0000,0,0,24160,211.3375,B5,S,2,"St Louis, MO"
1,1,1,male,0.9167,1,2,113781,151.5500,C22 C26,S,11,"Montreal, PQ / Chesterville, ON"
2,1,0,female,2.0000,1,2,113781,151.5500,C22 C26,S,0,"Montreal, PQ / Chesterville, ON"
3,1,0,male,30.0000,1,2,113781,151.5500,C22 C26,S,0,"Montreal, PQ / Chesterville, ON"
4,1,0,female,25.0000,1,2,113781,151.5500,C22 C26,S,0,"Montreal, PQ / Chesterville, ON"


In [4]:
#I intentionally commented th convert_object out and decided to create a function that will help
def handle_non_numeric_data(df):
    colums = df.columns.values
    for column in colums:
        text_digit_vals ={}
        
        def convert_to_int(val):
            return text_digit_vals[val]
        
        if df[column].dtype != np.int64 and df[column].dtype != np.float64:
            column_contents = df[column].values.tolist()
            unique_elements = set(column_contents)
            x=0
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    text_digit_vals[unique] = x
                    x+=1
            df[column] = list(map(convert_to_int, df[column]))
            
    return df

df = handle_non_numeric_data(df)

df.head()

,pclass,survived,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,home.dest
0,1,1,1,29.0000,0,0,751,211.3375,104,2,1,2
1,1,1,0,0.9167,1,2,503,151.5500,138,2,22,195
2,1,0,1,2.0000,1,2,503,151.5500,138,2,0,195
3,1,0,0,30.0000,1,2,503,151.5500,138,2,0,195
4,1,0,1,25.0000,1,2,503,151.5500,138,2,0,195


In [6]:
#Smiles! I decided to use Kmeans for prediction
df.drop(['ticket'],1, inplace=True)
X = np.array(df.drop(['survived'], 1).astype(float))
X = preprocessing.scale(X)
y = np.array(df['survived'])

clf= KMeans(n_clusters=2)
clf.fit(X)

correct=0
for i in range(len(X)):
    predict_me=np.array(X[i].astype(float))
    predict_me = predict_me.reshape(-1, len(predict_me))
    prediction = clf.predict(predict_me)
    if prediction[0] ==y[i]:
        correct+=1
        
        
print(correct/len(X))

0.7043544690603514


In [11]:
#That's high but not too high
#Let me use Logistic Regression and see how it goes
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)
print('X_train', X_train.size)
print('X_test', X_test.size)
print('y_train', y_train.size)
print('y_test', y_test.size)
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
model = clf.fit(X_train, y_train)
prediction = model.predict(X_test)

accuracy = clf.score(X_test, y_test)
print (accuracy)

X_train 10470
X_test 2620
y_train 1047
y_test 262
0.9618320610687023


# The next is K Nearest Neighbors

In [13]:
from sklearn import neighbors, svm
clf=neighbors.KNeighborsClassifier()
model = clf.fit(X_train, y_train)
predi = model.predict(X_test)
accuracy_score(y_test, predi)

0.9007633587786259

In [14]:
#Not up to the mark

# Using Support Vector Machine

In [15]:
clf=svm.SVC()
clf.fit(X_train,y_train)
accuracy = clf.score(X_test, y_test)
print(accuracy)

0.9656488549618321


I made it!!!

Thanks at SCA! I just Learnt this today in the 70days ml prebootcamp. Actually, I have been trying to do it but didn't know
how to drop the strings in most of the columns. I am happy I learn't it. I promise to do better if I get a mentor